# 2018 Squirrel Census

## Part 1: Database and Jupyter Notebook Set Up

Import the dataset with `mongoimport --type csv -d squirrels_db -c squirrels --headerline  --drop 2018_Central_Park_Squirrel_Census_-_Squirrel_Data.csv`

In [1]:
# Import dependencies
from pymongo import MongoClient
import requests 
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'squirrels_db', 'test', 'travel_db', 'uk_food']


In [4]:
# assign the squirrels_db database to a variable name
db = mongo['squirrels_db']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['squirrels']


In [6]:
# assign the collection to a variable
squirrels = db['squirrels']

In [7]:
# review a document in the census_2018 collection
pprint(db.squirrels.find_one())

{'Above_Ground_Sighter_Measurement': '',
 'Age': '',
 'Approaches': 'FALSE',
 'Chasing': 'FALSE',
 'Climbing': 'FALSE',
 'Color_notes': '',
 'Combination_of_Primary_and_Highlight_Color': '+',
 'Date': 10142018,
 'Eating': 'FALSE',
 'Foraging': 'FALSE',
 'Hectare': '37F',
 'Hectare_Squirrel_Number': 3,
 'Highlight_Fur_Color': '',
 'Indifferent': 'FALSE',
 'Kuks': 'FALSE',
 'Lat/Long': 'POINT (-73.9561344937861 40.7940823884086)',
 'Location': '',
 'Moans': 'FALSE',
 'Other_Activities': '',
 'Other_Interactions': '',
 'Primary_Fur_Color': '',
 'Quaas': 'FALSE',
 'Running': 'FALSE',
 'Runs_from': 'FALSE',
 'Shift': 'PM',
 'Specific_Location': '',
 'Tail_flags': 'FALSE',
 'Tail_twitches': 'FALSE',
 'Unique_Squirrel_ID': '37F-PM-1014-03',
 'X': -73.95613449,
 'Y': 40.79408239,
 '_id': ObjectId('642cca8711739165e7ee2b2f')}


## Part 2: Review and update the database

In [8]:
# Total number of documents
print("Number of documents: ", squirrels.count_documents({}))

Number of documents:  3023


In [10]:
# Total number of documents
print("Number of documents: ", squirrels.count_documents({}))

# Total number of unique squirrels
unqiue_squirrels = squirrels.distinct("Unique_Squirrel_ID")
print("Number of unique squirrels in data set is: ",len(unqiue_squirrels))

Number of documents:  3023
Number of unique squirrels in data set is:  3018


In [13]:
# Looking into the squirrels with duplicate 'Unique Squirrel ID's'
query = [{'$group': {'_id': "$Unique_Squirrel_ID", 'count': { '$sum': 1 }}}]

results = list(squirrels.aggregate(query))

# Save results to a dataframe
unique_df = pd.DataFrame(results)

# filter dataframe to only those that have a count greater than two
dups_df = unique_df.loc[unique_df['count'] >= 2]
dups_df


,_id,count
102,1F-AM-1010-04,2
146,40B-AM-1019-06,2
767,7D-PM-1010-01,2
2400,4C-PM-1010-05,2
2917,37E-PM-1006-03,2


In [14]:
# Looking into data that have the same 'Unique Squirrel ID'

# Filter dataset to 5 unique squirrel ID's that have duplicates
query = {'Unique_Squirrel_ID' : {'$in': ["1F-AM-1010-04", "40B-AM-1019-06", "7D-PM-1010-01", "4C-PM-1010-05", "37E-PM-1006-03"]}}
results = list(squirrels.find(query))

# Save to a datafame
duplicate_squirrels_df = pd.DataFrame(results)

# Sort DataFrame on 'Unique Squirrel ID'
duplicate_squirrels = duplicate_squirrels_df.sort_values(by=['Unique_Squirrel_ID'])
duplicate_squirrels

,_id,X,Y,Unique_Squirrel_ID,Hectare,Shift,Date,Hectare_Squirrel_Number,Age,Primary_Fur_Color,...,Kuks,Quaas,Moans,Tail_flags,Tail_twitches,Approaches,Indifferent,Runs_from,Other_Interactions,Lat/Long
7,642cca8711739165e7ee3514,-73.976620,40.766191,1F-AM-1010-04,01F,AM,10102018,4,Adult,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,,POINT (-73.9766195697096 40.7661907261547)
8,642cca8711739165e7ee3564,-73.976588,40.766095,1F-AM-1010-04,01F,AM,10102018,4,Adult,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,,POINT (-73.9765877920583 40.7660946071574)
0,642cca8711739165e7ee2b97,-73.957044,40.794851,37E-PM-1006-03,37E,PM,10062018,3,Adult,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,me,POINT (-73.9570437717691 40.794850940803904)
6,642cca8711739165e7ee340e,-73.956967,40.794879,37E-PM-1006-03,37E,PM,10062018,3,Adult,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,me,POINT (-73.9569670900695 40.794879285681304)
2,642cca8711739165e7ee3133,-73.957711,40.798406,40B-AM-1019-06,40B,AM,10192018,6,Juvenile,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,,POINT (-73.957710975899 40.7984055614047)
4,642cca8711739165e7ee33a6,-73.958236,40.798775,40B-AM-1019-06,40B,AM,10192018,6,Juvenile,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,,POINT (-73.9582363437135 40.7987752300505)
3,642cca8711739165e7ee3369,-73.977262,40.769282,4C-PM-1010-05,04C,PM,10102018,5,Adult,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,,POINT (-73.9772624834791 40.7692821918543)
9,642cca8711739165e7ee3677,-73.977165,40.769305,4C-PM-1010-05,04C,PM,10102018,5,Adult,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,,POINT (-73.9771646835311 40.7693049165084)
1,642cca8711739165e7ee2d34,-73.974945,40.772066,7D-PM-1010-01,07D,PM,10102018,1,Adult,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,,POINT (-73.9749446629706 40.7720659696067)
5,642cca8711739165e7ee33e9,-73.974852,40.771959,7D-PM-1010-01,07D,PM,10102018,1,Adult,Gray,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,,POINT (-73.9748522796455 40.7719586621003)


The data for each unqiue squirrel Id that has a duplicate is the same, except for the longitude and latitude values.  Which are slightly different between the squirrels with the same unique ID
Given I'm unable to query the data, I will keep the duplicates in given their size is rather minimal and I do not have a clear reason to exlude them at this point.

<h5>Primary Fur Color

In [15]:
# Checking the primary fur colour field
query = [{'$group': {'_id': "$Primary_Fur_Color", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)


[{'_id': 'Cinnamon', 'count': 392},
 {'_id': '', 'count': 55},
 {'_id': 'Gray', 'count': 2473},
 {'_id': 'Black', 'count': 103}]


In [16]:
# Primary Fur Color - updating fields with "" to 'Unknown'
query = {"Primary_Fur_Color" : ""}
new_values = {"$set": {"Primary_Fur_Color": "Not_noted"}}
squirrels.update_many(query, new_values)

# Checking the update worked
query = [{'$group': {'_id': "$Primary_Fur_Color", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)


[{'_id': 'Not_noted', 'count': 55},
 {'_id': 'Black', 'count': 103},
 {'_id': 'Gray', 'count': 2473},
 {'_id': 'Cinnamon', 'count': 392}]


<h5> Age

In [17]:
# Checking the age field
query = [{'$group': {'_id': "$Age", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)


[{'_id': '', 'count': 121},
 {'_id': 'Adult', 'count': 2568},
 {'_id': 'Juvenile', 'count': 330},
 {'_id': '?', 'count': 4}]


In [18]:
# Age - updating fields with '' to 'Unknown'
query = {"Age" : ''}
new_values = {"$set": {"Age": "Not_noted"}}
squirrels.update_many(query, new_values)

# Age - updating fields with '?' to 'Unknown'
query = {"Age" : '?'}
new_values = {"$set": {"Age": "Not_noted"}}
squirrels.update_many(query, new_values)

# Checking the update worked
query = [{'$group': {'_id': "$Age", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)


[{'_id': 'Not_noted', 'count': 125},
 {'_id': 'Juvenile', 'count': 330},
 {'_id': 'Adult', 'count': 2568}]


<h5> Shift field

In [19]:
# Checking the shift field
query = [{'$group': {'_id': "$Shift", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)

[{'_id': 'PM', 'count': 1676}, {'_id': 'AM', 'count': 1347}]


<h5> Hectare

In [20]:
# Checking the Hectare field
query = [{'$group': {'_id': "$Hectare", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)

[{'_id': '10F', 'count': 15},
 {'_id': '08C', 'count': 10},
 {'_id': '35G', 'count': 3},
 {'_id': '39F', 'count': 9},
 {'_id': '41G', 'count': 2},
 {'_id': '09E', 'count': 7},
 {'_id': '19E', 'count': 5},
 {'_id': '22F', 'count': 15},
 {'_id': '08F', 'count': 8},
 {'_id': '22A', 'count': 9},
 {'_id': '28C', 'count': 4},
 {'_id': '09G', 'count': 10},
 {'_id': '31C', 'count': 4},
 {'_id': '22E', 'count': 2},
 {'_id': '04E', 'count': 12},
 {'_id': '33F', 'count': 8},
 {'_id': '07C', 'count': 9},
 {'_id': '13A', 'count': 9},
 {'_id': '01H', 'count': 4},
 {'_id': '30H', 'count': 4},
 {'_id': '37D', 'count': 10},
 {'_id': '32H', 'count': 4},
 {'_id': '18I', 'count': 12},
 {'_id': '14B', 'count': 11},
 {'_id': '23A', 'count': 5},
 {'_id': '14F', 'count': 15},
 {'_id': '36H', 'count': 4},
 {'_id': '36A', 'count': 11},
 {'_id': '39D', 'count': 10},
 {'_id': '35C', 'count': 14},
 {'_id': '42A', 'count': 2},
 {'_id': '42G', 'count': 6},
 {'_id': '32E', 'count': 30},
 {'_id': '08B', 'count': 18},


<h5> Location

In [21]:
# Checking the Location field
query = [{'$group': {'_id': "$Location", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)

[{'_id': '', 'count': 64},
 {'_id': 'Above Ground', 'count': 843},
 {'_id': 'Ground Plane', 'count': 2116}]


In [22]:
# Location - updating fields with '' to 'Unknown'
query = {"Location" : ''}
new_values = {"$set": {"Location": "Not_noted"}}
squirrels.update_many(query, new_values)

# Checking the update worked
query = [{'$group': {'_id': "$Location", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)

[{'_id': 'Above Ground', 'count': 843},
 {'_id': 'Ground Plane', 'count': 2116},
 {'_id': 'Not_noted', 'count': 64}]


<h5> Combinatin of Primary and Highlight Color

In [24]:
# Checking the combination of primary and highligh color field
query = [{'$group': {'_id': "$Combination_of_Primary_and_Highlight_Color", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)

[{'_id': 'Cinnamon+White', 'count': 94},
 {'_id': 'Black+White', 'count': 2},
 {'_id': 'Gray+Cinnamon', 'count': 752},
 {'_id': 'Gray+Cinnamon, White', 'count': 265},
 {'_id': 'Gray+Black, White', 'count': 7},
 {'_id': 'Black+Cinnamon, White', 'count': 3},
 {'_id': 'Gray+Black', 'count': 24},
 {'_id': '+', 'count': 55},
 {'_id': 'Cinnamon+Black, White', 'count': 3},
 {'_id': 'Black+Gray, White', 'count': 1},
 {'_id': 'Cinnamon+Gray', 'count': 162},
 {'_id': 'Gray+', 'count': 895},
 {'_id': 'Black+Cinnamon', 'count': 15},
 {'_id': 'Cinnamon+', 'count': 62},
 {'_id': 'Black+', 'count': 74},
 {'_id': 'Gray+Black, Cinnamon, White', 'count': 32},
 {'_id': 'Cinnamon+Gray, White', 'count': 58},
 {'_id': 'Gray+White', 'count': 489},
 {'_id': 'Gray+Black, Cinnamon', 'count': 9},
 {'_id': 'Cinnamon+Black', 'count': 10},
 {'_id': 'Black+Gray', 'count': 8},
 {'_id': 'Cinnamon+Gray, Black', 'count': 3}]


<h5> Highlight Fur Color

In [25]:
query = [{'$group': {'_id': "$Highlight_Fur_Color", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)

[{'_id': 'Gray, White', 'count': 59},
 {'_id': 'Cinnamon', 'count': 767},
 {'_id': 'White', 'count': 585},
 {'_id': 'Cinnamon, White', 'count': 268},
 {'_id': 'Gray, Black', 'count': 3},
 {'_id': 'Black', 'count': 34},
 {'_id': 'Black, Cinnamon, White', 'count': 32},
 {'_id': '', 'count': 1086},
 {'_id': 'Black, Cinnamon', 'count': 9},
 {'_id': 'Black, White', 'count': 10},
 {'_id': 'Gray', 'count': 170}]


In [26]:
# highlight fur color - updating fields with '' to 'not_noted'
query = {"Highlight_Fur_Color" : ''}
new_values = {"$set": {"Highlight_Fur_Color": "Not_noted"}}
squirrels.update_many(query, new_values)

# Checking the update worked
query = [{'$group': {'_id': "$Highlight_Fur_Color", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)

[{'_id': 'Black, Cinnamon', 'count': 9},
 {'_id': 'Black', 'count': 34},
 {'_id': 'White', 'count': 585},
 {'_id': 'Gray, White', 'count': 59},
 {'_id': 'Cinnamon, White', 'count': 268},
 {'_id': 'Gray', 'count': 170},
 {'_id': 'Cinnamon', 'count': 767},
 {'_id': 'Gray, Black', 'count': 3},
 {'_id': 'Black, White', 'count': 10},
 {'_id': 'Not_noted', 'count': 1086},
 {'_id': 'Black, Cinnamon, White', 'count': 32}]


<h5> Specific Location

In [27]:
query = [{'$group': {'_id': "$Specific_Location", 'count': { '$sum': 1 }}}]
results = list(squirrels.aggregate(query))
pprint(results)

[{'_id': 'on fallen tree', 'count': 1},
 {'_id': 'High in a tree', 'count': 1},
 {'_id': 'Different tree', 'count': 1},
 {'_id': 'in trees, then to ground', 'count': 1},
 {'_id': 'Climbing up rocks', 'count': 1},
 {'_id': 'by water', 'count': 1},
 {'_id': 'Tree', 'count': 7},
 {'_id': 'On rocks', 'count': 1},
 {'_id': 'on hill', 'count': 1},
 {'_id': 'fence', 'count': 5},
 {'_id': 'Running across fence between statue & swings', 'count': 1},
 {'_id': 'on path', 'count': 1},
 {'_id': 'Fence pole', 'count': 1},
 {'_id': 'Dog chased up tree', 'count': 1},
 {'_id': 'Low tree', 'count': 1},
 {'_id': 'bench', 'count': 1},
 {'_id': 'Running from one tree to the other', 'count': 1},
 {'_id': 'Oak tree near rock outcropping', 'count': 1},
 {'_id': 'Behind fence, tree trunk', 'count': 1},
 {'_id': 'on branch of dead tree', 'count': 1},
 {'_id': 'Tree & ground', 'count': 2},
 {'_id': 'perched on small fence', 'count': 1},
 {'_id': 'on a log', 'count': 1},
 {'_id': 'unsure, just heard kuks', 'count

## Add new latitude and longitude and round to 2 decimal places